In [ ]:
!pip install boto3 botocore torch-model-archiver

In [ ]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_ENDPOINT_URL_S3')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

In [ ]:
work_dir=os.getcwd()
print(f"Current working directory: {work_dir}")

In [ ]:
tuned_model_dir = f"{work_dir}/images/model-tuned"
print(f"Local tuned model dir: {tuned_model_dir}")

In [ ]:
minio_console=!oc get route minio-console -o jsonpath='{.spec.host}'
print(f"Minio console: https://{minio_console.n}/browser/{bucket_name}")

In [ ]:
# This can take 10-15mins to zip up

!(cd {tuned_model_dir} && zip -r {work_dir}/model/model.zip *)

In [ ]:
!(cd {work_dir}/model && torch-model-archiver --model-name stable-diffusion --version 1.0 --handler stable_diffusion_handler.py --extra-files model.zip -r requirements.txt -f)

In [ ]:
if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or data connection variables are empty.  "
                     "Please check your data connection to an S3 bucket.")

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)

def upload_directory_to_s3(local_directory, s3_prefix):
    num_files = 0
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)
            num_files += 1
    return num_files


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [ ]:
list_objects("models")

In [ ]:
model_dir = work_dir + "/model"

if not os.path.isdir(model_dir):
    raise ValueError(f"The directory '{model_dir}' does not exist.  "
                     "Did you finish training the model in the previous notebook?")

num_files = upload_directory_to_s3(model_dir, "models")

if num_files == 0:
    raise ValueError(f"No files uploaded. Check the {model_dir} directory.")

In [ ]:
list_objects("models")